<a href="https://colab.research.google.com/github/LukasEder1/CKE-Evaluation/blob/master/CreateKeywordCollections.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone "https://github.com/LukasEder1/CKE-Evaluation.git"
%cd "/content/CKE-Evaluation"

Cloning into 'CKE-Evaluation'...
remote: Enumerating objects: 34, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 34 (delta 5), reused 15 (delta 0), pack-reused 0
Unpacking objects: 100% (34/34), 170.52 KiB | 2.13 MiB/s, done.
/content/CKE-Evaluation


In [2]:
%pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/boudinfl/pke.git to /tmp/pip-req-build-zsaulavc
  Running command git clone --filter=blob:none --quiet https://github.com/boudinfl/pke.git /tmp/pip-req-build-zsaulavc
  Resolved https://github.com/boudinfl/pke.git to commit 8f1d05dcc52041c9920ba0f9d5231fe6086d12c4
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.3/97.3 KB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 77.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 11.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.2/60.2 KB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━

NotImplementedError: ignored

# Keyword Collection Creation

In [4]:
from contrastive_keyword_extraction import contrastive_extraction, final_score
import sqlite3
import pandas as pd
from baselines import *
from tqdm import tqdm
import string
import pickle
import sentence_comparision
import sentence_importance
import summary
import utilities
import keyword_extraction
import nltk

nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [5]:
with open("policy_data.pkl", "rb") as file:
  policies = pickle.load(file)

In [6]:
def create_collection(sites, 
                      ke_extractor = keyword_extraction.extract_yake, 
                      num_keywords=10,
                      max_ngram=2, 
                      sentence_matcher = sentence_comparision.match_sentences_semantic_search,
                      importance_estimator = sentence_importance.text_rank_importance,
                      use_furthest=False, 
                      name_prefix="",
                      make_data_persistent=False, 
                      path="dataframes",
                      threshold=0.6,
                      stopwords=[],
                      combinator=utilities.alpha_combination,
                      gamma = 0.5,
                      num_splits=1,
                      matching_model="all-MiniLM-L6-v2"):
    
    for i in tqdm(sites.keys()):

        documents = sites[i]
        
        # run CKE-pipeline
        # Extract Keywords, and Matched sentences
        keywords, matched_dict, changed_indices, additions, deletions, new_indices, ranking, removed, matched_indices, unified_delitions = contrastive_extraction(documents, 
                                                                            max_ngram=max_ngram,
                                                                            min_ngram=1, 
                                                                            show_changes=False, 
                                                                            symbols_to_remove=string.punctuation,
                                                                            importance_estimator=importance_estimator,
                                                                            match_sentences=sentence_matcher,
                                                                            threshold=threshold,
                                                                            extra_stopwords=stopwords,
                                                                            top_k=int(num_splits),
                                                                            combinator=combinator,
                                                                            alpha_gamma=gamma,
                                                                            matching_model=matching_model)
        
        

    
        # create itermediate
        kws, scores = extract_from_dict(keywords)
        
        pipeline_frame = pd.DataFrame({'keyword': kws, 'score': scores})
        
        #extractor = lambda x: ke_extractor(x, max_ngram_size=max_ngram, numOfKeywords=num_keywords)
        
        # create CKE on the specified baseline
        baseline_keywords = baseline_diff_content(additions, unified_delitions, ke_extractor, num_keywords, max_ngram)
        
        baseline_kws, baseline_scores = extract_from_tuple_list(baseline_keywords)
        
        baseline_frame1 = pd.DataFrame({'keyword': baseline_kws, 'score': baseline_scores})
        
        
        # create CKE for baseline method 2
        baseline_keywords2 = baseline_keywords_in_diff(documents, ke_extractor, additions, deletions, candidates=50, max_ngram=max_ngram)
        
        baseline_kws2, baseline_scores2 = extract_from_dict(baseline_keywords2)
        
        baseline_frame2 = pd.DataFrame({'keyword': baseline_kws2, 'score': baseline_scores2})
        
        # Baseline 3
        baseline_keywords3 = baseline3(documents, additions, unified_delitions, max_ngram)
        
        baseline_kws3, baseline_scores3 = extract_from_dict(baseline_keywords3)
        
        baseline_frame3 = pd.DataFrame({'keyword': baseline_kws3, 'score': baseline_scores3})

        # Baseline 4
        baseline_keywords4 = baseline4(documents, max_ngram, stopwords)
        
        baseline_kws4, baseline_scores4 = extract_from_dict(baseline_keywords4)
        
        baseline_frame4 = pd.DataFrame({'keyword': baseline_kws4, 'score': baseline_scores4})

        # decide, wether to actually save the data
        if make_data_persistent:
            
            pipeline_frame.to_csv(f"{path}/{name_prefix}_pipeline_keywords_{i}.csv", index=False)
            
            baseline_frame1.to_csv(f"{path}/{name_prefix}_baseline1_keywords_{i}.csv", index=False)
            
            baseline_frame2.to_csv(f"{path}/{name_prefix}_baseline2_keywords_{i}.csv", index=False)
            
            baseline_frame3.to_csv(f"{path}/{name_prefix}_baseline3_keywords_{i}.csv", index=False)
            
            baseline_frame4.to_csv(f"{path}/{name_prefix}_baseline4_keywords_{i}.csv", index=False)
    
    return pipeline_frame, baseline_frame1, baseline_frame2, baseline_frame3, baseline_frame4

In [7]:
create_collection(policies, 
                  ke_extractor = keyword_extraction.extract_yake, 
                  num_keywords=15,
                  max_ngram=2, 
                  sentence_matcher = sentence_comparision.match_sentences_semantic_search,
                  importance_estimator = sentence_importance.text_rank_importance,
                  use_furthest=False, 
                  name_prefix="standard",
                  make_data_persistent=True, 
                  path="Dataframes",
                  threshold=0.65,
                  stopwords=nltk.corpus.stopwords.words("english"),
                  combinator=utilities.alpha_combination,
                  gamma = 0.5,
                  num_splits=1,
                  matching_model ="msmarco-distilbert-base-v4")

  0%|          | 0/25 [00:00<?, ?it/s]

100%|██████████| 25/25 [04:26<00:00, 10.66s/it]


(               keyword     score
 0                  app  0.132400
 1               mobile  0.057117
 2           mobile app  0.057117
 3                  may  0.037800
 4              certain  0.037578
 5           additional  0.031150
 6               choice  0.031150
 7               within  0.031150
 8               listed  0.028183
 9                  use  0.022545
 10           available  0.021473
 11             provide  0.020766
 12               pages  0.020668
 13            services  0.020131
 14          collection  0.019539
 15                 via  0.018789
 16    certain services  0.018789
 17      services pages  0.018789
 18        pages listed  0.018789
 19          listed may  0.018789
 20       may available  0.018789
 21       available via  0.018789
 22          via mobile  0.018789
 23              notice  0.015575
 24             options  0.015575
 25         may provide  0.015575
 26  provide additional  0.015575
 27   additional notice  0.015575
 28       noti

In [48]:
from itertools import product


def cartesian_product(params):
    
    # gett all possible combinations
    return list(product(*params.values()))



# Parameters to Use

In [49]:
parameters = {"matcher": [sentence_comparision.match_sentences_semantic_search,
                         sentence_comparision.match_sentences_tfidf_weighted],
              
             "ie": [sentence_importance.text_rank_importance,
                    sentence_importance.yake_weighted_importance],
              
             "threshold": [0.5, 0.6, 0.7]
              
             }


In [70]:
combinations = cartesian_product(parameters)
num_of_combinations = len(combinations)

In [53]:
def create_cartesian_collection(params, 
                                sites, 
                                baseline_ke_extractor = keyword_extraction.extract_yake, 
                                num_keywords=10, 
                                max_ngram=2, 
                                make_data_persistent=False,
                                file_prefix = "combination",
                                path="Dataframes",
                                compare_k = 15):
            
        
    combinations = cartesian_product(params)
    
    number_of_combinations = len(combinations)
    
    count = 0
    
    for combination in combinations:
        
        matcher, ie, threshold = combination
        
        print(f"Contrastive Keyword Extraction pipeline is being ran with combination {count}:")
        
        pipeline_frame, baseline_frame1, baseline_frame2, baseline_frame3, baseline_frame4 = create_collection(sites = sites, 
                                                                  ke_extractor = baseline_ke_extractor, 
                                                                  num_keywords = num_keywords, 
                                                                  max_ngram = max_ngram, 
                                                                  sentence_matcher = matcher,
                                                                  importance_estimator = ie,
                                                                  name_prefix=f"{file_prefix}_{count}",
                                                                  make_data_persistent=make_data_persistent,
                                                                  path=path,
                                                                  threshold=threshold)
        
      
        
        count += 1
        

In [56]:
create_cartesian_collection(parameters,
                            policies,
                            baseline_ke_extractor = keyword_extraction.extract_yake,
                            num_keywords=15,
                            max_ngram=2,
                            make_data_persistent=True,
                            file_prefix="combination",
                            path="Dataframes")

Contrastive Keyword Extraction pipeline is being ran with combination 0:


  0%|          | 0/25 [00:00<?, ?it/s]

100%|██████████| 25/25 [03:55<00:00,  9.41s/it]


Contrastive Keyword Extraction pipeline is being ran with combination 1:


100%|██████████| 25/25 [04:06<00:00,  9.87s/it]


...

100%|██████████| 25/25 [03:51<00:00,  9.25s/it]


Contrastive Keyword Extraction pipeline is being ran with combination 10:


100%|██████████| 25/25 [03:53<00:00,  9.35s/it]


Contrastive Keyword Extraction pipeline is being ran with combination 11:


100%|██████████| 25/25 [03:54<00:00,  9.39s/it]


# Evaluate Results


## Compare against baselines

In [17]:
summary.extensive_summary(list(policies.keys()),
                          show_results=True, 
                          k=15,
                          name_a = f"standard_pipeline_keywords", 
                          name_b = f"standard_baseline1_keywords",
                          save_prefix=f"standard_", 
                          path='Dataframes')


,Site,F1,Precision,Recall,IoU,#overlaps
0,90536,0.000000,0.000000,0.000000,0.000000,0
1,90344,0.435556,0.466667,0.466667,0.333333,7
2,98640,0.000000,0.000000,0.000000,0.000000,0
3,98585,0.035556,0.133333,0.133333,0.071429,2
4,99880,0.533333,0.533333,0.533333,0.363636,8
5,108079,0.320000,0.400000,0.400000,0.250000,6
6,90555,0.000000,0.000000,0.000000,0.000000,0
7,90545,0.080000,0.200000,0.200000,0.111111,3
8,98553,0.320000,0.400000,0.400000,0.272727,6
9,98572,0.008889,0.066667,0.066667,0.034483,1


## Manual Policy Keywords

In [18]:
manual = {
    "90536": ["TRUSTe's",
            "European Union", "day trial",
            "personally identifiable", "personally", "Salesforce",
            "identifiable information", "testimonial", "gifs"
            "version", "Third Party", "clear gifs", "web beacons",
            "service providers", "email"],


    "90344": ["Telerik", "May", "Progress Software", "software", "web", "site",
         "community progress", "Telerik Analytics",
        "provider", "service providers", "geo-location", "ads", "Google",
         "google Analytics", "web sites", "information", "children"],


    "98640": ["validation page", "scripts", "HTML5", "tools", "management tools", "management",
         "various", "various browsers", "removing", "transparency", "may offer"],   


    "98585": ["center", "center cloud", "cloud console", "security center", "console", "cloud",
          "solution", "onedrive", "web beacons"], 




    "99880": ["science", "reproductive science", "Michele Abbott", "Michele", "Abbott"
    "electronic", "health information", "insurance company", "health"],




    "108079": ["Insights", "viva","club","sydney","Viva Sydney","Sydney Club", "visitors", "large"], 



    "90555":  ["ads", "google", "use", "privacy policy", "policy", "privacy", "data", "data control"],


    "90545":  ["Yahoo", "Privacy Policy", "AOL", "Oath", "choices", "your choices",
          "privacy", "oath privacy"],



    "98553": ["April", "2016", "affiliated websites", "providers", "billing information",
    "over-the-phone", "advertisements", "plans", "partners"], 


    "98572": ["job applications", "Privacy Shield", "Shield Framework", "framework", "shield",
          "secure data", "data recovery",  "Truste"],



    "98659": ["affiliated entities", "TOS", "without limitation",
    "Seed Spark", "speed", "spark", "liable" ,"opt","opt out", "google",
          "remarketing","updated",  "California", "vendors"],


    "98706": ["Australian", "Learning", "online advertising", "New Zealand", "United States",
           "America",  "Melbourne", "contact us", "contact"],


    "108052":  ["Reloaded", "Reloaded Games", "games", "K2", "K2 network", "January"],


    "108097": ["Website Security", "PKI solutions", "DigiCert", "DigiCert Inc" "MySymantec",  "Global",
            "transfers",
            "approved partners", "service providers", "contractors", "around the world"], 


    100541: ["live", "browse history", "Synchrony Bank", "Synchrony", "GE money", "money bank", "GEMB",
         "Credit Card", "Fair Credit", "Billing Act"],

    108771: ["tap", "ALTERNATIVE PRESS LLC", "alternative", "press", "llc", "alternative press", "press llc"],

    2435: ["properties", "sites", "requests", "california", "545", "5215", "john", "carpenter", "freeway", 
        "john carpenter" ,"700", "1400", "Connor", "Blvd", "Connor Blvd"],

    100595: ["evine", "shopnbc", "ge money", "synchrony", "mobile", "mobile device", "money bank", "Synchrony Bank",
         "california", "TLS"],

    108778: ["DART", "DART cookie", "parties", "third party", "email"], # email is written differently 
    100419: ["toxic shock", "toxic", "shock", "shockya", "advertising", "third-party", "DART",
         "web", "site", "ads"],



    108438: ["TakeLessons", "profile", "public", "updates", "changes", "unsubscribe"], 

    108835: ["personally identifiable", " identifiable information", "cookies", "use", "personally",
             "identifiable"],

    # small change examples
    106348: ["advertising", "www", "targeted", "partner", "ad network", "ad", "network"],
    90041: ["mobile", "mobile app", "app", "may"],
    106486: ["google", "cookies", "13", "Crosby St", "Hollywood", "895", "use", "opt"]
          
          
         }

## Compare Against Manually Annotated Data

In [57]:
import string
import nltk

def remove_punctuation(text, symbols=string.punctuation):
    return "".join([char for char in text if char not in symbols])

stop_words = nltk.corpus.stopwords.words("english")

In [58]:
def process_text(string):
    string = remove_punctuation(string)
    words_to_use = []
    for word in nltk.word_tokenize(string):
        if word not in stop_words:
            words_to_use.append(word)
        
    return " ".join(words_to_use)

In [59]:
manual = {site: [process_text(word) for word in words] for site, words in manual.items()}

In [60]:
def evaluate_manual(manual, compare="pipeline", typ="manual", path = "Dataframes"):
    f1s = []
    precs = []
    recalls = []
    overlaps = []
    
    num_manual = []
    stemmer = nltk.stem.PorterStemmer()
    sites = list(manual.keys())
    
    for site in sites:
        
        kws = manual[site]
        
        kws_hat = list(pd.read_csv(f"{path}/{typ}_{compare}_keywords_{site}.csv").keyword)
        
        kws = [stemmer.stem(kw) for kw in kws]

        kws_hat = [stemmer.stem(str(kw_hat)) for kw_hat in kws_hat][:max(len(kws), 10)]
        
        overlap = summary.number_of_overlaps(set(kws), set(kws_hat))
        
        recall = overlap / max(1, len(kws_hat))

        precision = overlap / max(1, len(kws))

        f1 =  2 * (precision * recall) / max(1, precision + recall)
        
        f1s.append(f1)
        precs.append(precision)
        recalls.append(recall)
        overlaps.append(overlap)
        num_manual.append(len(kws))
        
    summary_frame = pd.DataFrame({'site_id': [int(site) for site in sites], 
                                  'f1': f1s, 'precision': precs,
                                  'recall': recalls, '#overlaps': overlaps,
                                 '#manual keywords': num_manual})
    
    return summary_frame
    

In [72]:
def describe_combinations(combinations, prefix):
  for i in range(len(combinations)):
    print(f"Results for combination {i}: {combinations[i]}")

    print("Pipeline Results: ")
    display(evaluate_manual(manual, compare="pipeline", typ=f"{prefix}_{i}").drop("site_id", axis=1).describe())

    print("Baseline1 Results: ")
    display(evaluate_manual(manual, compare="baseline1", typ=f"{prefix}_{i}").drop("site_id", axis=1).describe())

    print("Baseline2 Results: ")
    display(evaluate_manual(manual, compare="baseline2", typ=f"{prefix}_{i}").drop("site_id", axis=1).describe())

    print("Baseline3 Results: ")
    display(evaluate_manual(manual, compare="baseline3", typ=f"{prefix}_{i}").drop("site_id", axis=1).describe())

    print("Baseline4 Results: ")
    display(evaluate_manual(manual, compare="baseline4", typ=f"{prefix}_{i}").drop("site_id", axis=1).describe())

## Results

In [73]:
describe_combinations(combinations, "combination")

Results for combination 0: (<function match_sentences_semantic_search at 0x7f1f04ef9280>, <function text_rank_importance at 0x7f2061f163a0>, 0.5)
Pipeline Results: 


,f1,precision,recall,#overlaps,#manual keywords
count,25.000000,25.000000,25.000000,25.000000,25.000000
mean,0.376601,0.499546,0.392228,4.200000,9.080000
std,0.218895,0.229745,0.150148,1.607275,3.161223
min,0.040000,0.200000,0.100000,1.000000,4.000000
25%,0.163265,0.333333,0.285714,3.000000,7.000000
50%,0.355556,0.444444,0.400000,4.000000,8.000000
75%,0.555556,0.625000,0.500000,5.000000,10.000000
max,0.727273,1.000000,0.727273,8.000000,17.000000


Baseline1 Results: 


,f1,precision,recall,#overlaps,#manual keywords
count,25.000000,25.000000,25.000000,25.000000,25.000000
mean,0.083807,0.138400,0.122857,1.240000,9.080000
std,0.166146,0.211071,0.164286,1.714643,3.161223
min,0.000000,0.000000,0.000000,0.000000,4.000000
25%,0.000000,0.000000,0.000000,0.000000,7.000000
50%,0.010204,0.071429,0.071429,1.000000,8.000000
75%,0.080000,0.142857,0.200000,2.000000,10.000000
max,0.705882,0.857143,0.600000,6.000000,17.000000


Baseline2 Results: 


...

,f1,precision,recall,#overlaps,#manual keywords
count,25.000000,25.000000,25.000000,25.000000,25.000000
mean,0.007651,0.021968,0.035048,0.200000,9.080000
std,0.017341,0.049928,0.076142,0.408248,3.161223
min,0.000000,0.000000,0.000000,0.000000,4.000000
25%,0.000000,0.000000,0.000000,0.000000,7.000000
50%,0.000000,0.000000,0.000000,0.000000,8.000000
75%,0.000000,0.000000,0.000000,0.000000,10.000000
max,0.057143,0.200000,0.250000,1.000000,17.000000


Baseline3 Results: 


,f1,precision,recall,#overlaps,#manual keywords
count,25.000000,25.000000,25.000000,25.000000,25.000000
mean,0.095907,0.144949,0.118886,1.280000,9.080000
std,0.194952,0.245884,0.180155,1.837571,3.161223
min,0.000000,0.000000,0.000000,0.000000,4.000000
25%,0.000000,0.000000,0.000000,0.000000,7.000000
50%,0.000000,0.000000,0.000000,0.000000,8.000000
75%,0.062284,0.176471,0.176471,2.000000,10.000000
max,0.705882,0.857143,0.600000,6.000000,17.000000


Baseline4 Results: 


,f1,precision,recall,#overlaps,#manual keywords
count,25.000000,25.000000,25.000000,25.000000,25.000000
mean,0.060623,0.106232,0.088661,1.040000,9.080000
std,0.130655,0.164932,0.134770,1.719496,3.161223
min,0.000000,0.000000,0.000000,0.000000,4.000000
25%,0.000000,0.000000,0.000000,0.000000,7.000000
50%,0.000000,0.000000,0.000000,0.000000,8.000000
75%,0.035556,0.142857,0.100000,1.000000,10.000000
max,0.457143,0.571429,0.411765,7.000000,17.000000
